In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

api_keys=[
    
     "  ",
     "  ",           #Enter your Custom Search API keys that you received from Google Cloud Console. You can get a key for each project. 👈👈👈
     "  ",           #10 projects can be created with a gmail account. 
     "  ",           #With the Custom Search API keys you will receive from these 10 gmail accounts, you can perform free SERP analysis for 1000 keywords.
     "  ",           #Example API KEY: AIzfWyB_c7f1e2Gz34543ffsdfj1LcUAD4QmzEtSJs
     "  ", 
     "  ", 
     "  ", 
     "  ", 
     "  "
]


cx = "     "    #Enter your search engine identifier (Search Engine ID) from Google Custom Search Engine (CSE) Example cx: 31b4785452sdfbea4af1 👈👈👈

def google_search(query, api_key, num_results=10):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        results = service.cse().list(q=query, cx=cx, num=num_results, cr="countryTR").execute()
        return results.get('items', [])
    except HttpError as e:
        if e.resp.status == 429:
            print(f"Rate limit exceeded with API Key: {api_key}")
            raise
        else:
            print(f"Error occurred with API Key: {api_key} {e}")
            raise

file_path = "    "   #👈👈👈 Enter the CSV file containing the words you want to analyze in SERP.
df = pd.read_csv(file_path)
search_terms = df['keywords'].tolist()
filtered_results_data = []
current_api_key_index = 0

search_term_index = 0
api_keys_exhausted = False

while search_term_index < len(search_terms):
    term = search_terms[search_term_index]
    try:
        print(f"Search Term ({search_term_index}): {term}")
        results = google_search(term, api_keys[current_api_key_index])
        rate = 1 
        for result in results:
            filtered_results_data.append({
                'Keyword': term,
                'Page': result.get('link'),
                'Meta Title': result.get('title'),
                'Meta Description': result.get('snippet'),
                'Rate': rate
            })
            rate += 1  
        search_term_index += 1
    except HttpError as e:
        if e.resp.status == 429:
            print(f"Rate limit exceeded with API Key: {api_keys[current_api_key_index]} Switching to another API key.")
            current_api_key_index = (current_api_key_index + 1) % len(api_keys)
            if current_api_key_index == 0:
                api_keys_exhausted = True
                print(f"All API keys have been used. You are stuck with the keyword: ({search_term_index}): {term}")
                break
            print(f"New API Key: {api_keys[current_api_key_index]}")
        else:
            print(f"Unexpected error occurred: {e}")
            break
            

csv_file = "      "  #👈👈👈 Enter the name of your CSV file where the results will be saved. Example: results.csv
df = pd.DataFrame(filtered_results_data)
df.to_csv(csv_file, index=False)

print("Data was saved to CSV.")